In [1]:

import time

import cv2
import numpy as np
import tensorflow as tf
from PIL import Image

from centernet import CenterNet

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

mode = "predict"
crop            = False
count           = False
heatmap_save_path = "model_data/heatmap_vision.png"

centernet = CenterNet(heatmap = True if mode == "heatmap" else False)


2022-12-01 01:44:24.567506: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 01:44:24.578661: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


model_data/centernet_resnet50_voc.h5 model, anchors, and classes loaded.
Configurations:
----------------------------------------------------------------------
|                     keys |                                   values|
----------------------------------------------------------------------
|               model_path |     model_data/centernet_resnet50_voc.h5|
|             classes_path |               model_data/voc_classes.txt|
|              input_shape |                               [512, 512]|
|                 backbone |                                 resnet50|
|               confidence |                                      0.3|
|                  nms_iou |                                      0.3|
|                      nms |                                     True|
|          letterbox_image |                                    False|
|                  heatmap |                                    False|
-----------------------------------------------------------

In [7]:
print(centernet.centernet.layers)

[<keras.engine.input_layer.InputLayer object at 0x7fa56ae6e7f0>, <keras.layers.convolutional.ZeroPadding2D object at 0x7fa56815a820>, <keras.layers.convolutional.Conv2D object at 0x7fa56815ad00>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fa568178040>, <keras.layers.core.activation.Activation object at 0x7fa568144310>, <keras.layers.pooling.MaxPooling2D object at 0x7fa568144d90>, <keras.layers.convolutional.Conv2D object at 0x7fa5680bd3a0>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fa5680bdf40>, <keras.layers.core.activation.Activation object at 0x7fa568144a30>, <keras.layers.convolutional.Conv2D object at 0x7fa56804e280>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fa568053790>, <keras.layers.core.activation.Activation object at 0x7fa568053e20>, <keras.layers.convolutional.Conv2D object at 0x7fa56805a580>, <keras.layers.convolutional.Conv2D object at 0x7fa56805ad00>, <keras.l

In [19]:
first_conv_layer = centernet.centernet.layers[2]
print(first_conv_layer)
print(first_conv_layer.weights)

[<tf.Variable 'conv1/kernel:0' shape=(7, 7, 3, 64) dtype=float32, numpy=
array([[[[ 1.94906741e-02, -2.80679278e-02,  7.65900407e-03, ...,
          -3.17537785e-02, -6.02737814e-02,  8.70463904e-03],
         [ 1.21029643e-02,  5.93803227e-02, -1.80919785e-02, ...,
          -3.90131362e-02,  2.31730919e-02,  1.54836066e-02],
         [ 2.25866819e-03, -5.08528687e-02, -2.90570073e-02, ...,
          -4.17485125e-02,  3.10440157e-02,  1.07532786e-02]],

        [[ 4.51345323e-03, -1.77525692e-02,  4.83121519e-04, ...,
          -1.80582870e-02, -1.72566436e-03,  1.92807689e-02],
         [ 1.55303208e-02,  7.53806159e-02, -1.63652170e-02, ...,
          -1.70739833e-02,  1.23554608e-02, -1.44846104e-02],
         [ 1.26479780e-02, -2.67485082e-02, -2.78822277e-02, ...,
          -1.99206714e-02, -2.04527173e-02, -5.49067045e-03]],

        [[ 2.00270489e-02, -2.21067891e-02,  9.86857060e-03, ...,
          -6.01443136e-03,  5.16742328e-03,  5.63815935e-03],
         [ 4.24814709e-02, 

In [25]:
import tensorflow as tf
image_input = tf.keras.layers.Input(shape=[512, 512, 3])
x = tf.keras.layers.ZeroPadding2D((3, 3))(image_input)

test_layer = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2), use_bias=False)(x)

my_model = tf.keras.Model(inputs=image_input, outputs=test_layer)
print(my_model)
print(my_model.layers[2])
print(my_model.layers[2].get_weights())

[array([[[[ 2.41471455e-03, -6.49734586e-03,  4.04764526e-02, ...,
           2.87743248e-02,  3.17485742e-02, -1.51973478e-02],
         [-2.98156720e-02, -3.67800072e-02,  1.72214545e-02, ...,
          -3.61760817e-02,  4.06847261e-02, -2.21715122e-03],
         [-3.96474898e-02,  1.33831277e-02, -3.11105456e-02, ...,
           2.97813229e-02, -2.72173211e-02,  2.96305381e-02]],

        [[ 8.85037333e-03,  2.86793225e-02,  4.19670083e-02, ...,
          -2.89099254e-02, -3.10514122e-04, -2.21125782e-04],
         [ 1.89589262e-02,  8.42147321e-03,  3.41728218e-02, ...,
           7.78391212e-03,  4.24216948e-02, -7.58424029e-03],
         [ 2.48045512e-02, -3.32441293e-02,  2.22857632e-02, ...,
          -5.08285314e-03,  1.55080371e-02, -9.86810401e-03]],

        [[-3.06850504e-02, -4.04737853e-02, -3.23623158e-02, ...,
           8.30298662e-03, -3.65263857e-02, -1.87158473e-02],
         [-1.14710666e-02,  2.30188034e-02, -3.88936549e-02, ...,
           1.43238157e-03,  2.980

In [21]:
my_model.layers[2].set_weights(first_conv_layer.get_weights())
my_model.layers[2].get_weights()

[array([[[[ 1.94906741e-02, -2.80679278e-02,  7.65900407e-03, ...,
           -3.17537785e-02, -6.02737814e-02,  8.70463904e-03],
          [ 1.21029643e-02,  5.93803227e-02, -1.80919785e-02, ...,
           -3.90131362e-02,  2.31730919e-02,  1.54836066e-02],
          [ 2.25866819e-03, -5.08528687e-02, -2.90570073e-02, ...,
           -4.17485125e-02,  3.10440157e-02,  1.07532786e-02]],
 
         [[ 4.51345323e-03, -1.77525692e-02,  4.83121519e-04, ...,
           -1.80582870e-02, -1.72566436e-03,  1.92807689e-02],
          [ 1.55303208e-02,  7.53806159e-02, -1.63652170e-02, ...,
           -1.70739833e-02,  1.23554608e-02, -1.44846104e-02],
          [ 1.26479780e-02, -2.67485082e-02, -2.78822277e-02, ...,
           -1.99206714e-02, -2.04527173e-02, -5.49067045e-03]],
 
         [[ 2.00270489e-02, -2.21067891e-02,  9.86857060e-03, ...,
           -6.01443136e-03,  5.16742328e-03,  5.63815935e-03],
          [ 4.24814709e-02,  7.92159066e-02, -8.25140160e-03, ...,
            8.542